[Recent Earthquakes in CSV format](https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php)

In [ ]:
%matplotlib inline
from pandas import read_csv

load = lambda x: read_csv(x, usecols=[
    'time', 'latitude', 'longitude', 'depth', 'mag', 'net', 'place',
], parse_dates=['time'])
try:
    recent_earthquakes = load('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.csv')
except IOError:
    recent_earthquakes = load('datasets/World-RecentEarthquakes.csv')
    
len(recent_earthquakes)

In [ ]:
recent_earthquakes.ix[recent_earthquakes.index[0]]

## How were the earthquakes distributed?

In [ ]:

recent_earthquakes.hist();

## What was the magnitude of the largest earthquake?

In [ ]:
recent_earthquakes['mag'].max()

## Which place had the largest earthquake?

In [ ]:
recent_earthquakes.ix[recent_earthquakes['mag'].idxmax()]['place']

## Plot earthquake magnitudes

In [ ]:
# Plot by day
recent_earthquakes_by_time = recent_earthquakes.set_index('time')
recent_earthquakes_by_time['mag'].plot();

In [ ]:
# Plot by week
recent_earthquakes_by_time.resample('W')['mag'].plot();

## How many earthquakes were north of the equator?

In [ ]:
len(recent_earthquakes[recent_earthquakes['latitude'] > 0])

## How many earthquakes were south of the equator?

In [ ]:
# Type your solution here and press CTRL-ENTER

## How many earthquakes did each sensor network measure?

In [ ]:
# Count earthquakes by sensor network code
recent_earthquakes['net'].value_counts()

## What were the names of each sensor network?

In [ ]:
# Load information on sensor networks
sensor_networks = read_csv('datasets/World-AdvancedNationalSeismicSystemNetworks.csv')
sensor_networks.head()

In [ ]:
# Rename columns
sensor_networks.rename(columns={
    'Network Code': 'net',
    'Network Name': 'netName',
}, inplace=True)
# Change codes to lowercase
sensor_networks['net'] = sensor_networks['net'].str.lower()
sensor_networks.head()

In [ ]:
# Merge tables
merged_earthquakes = recent_earthquakes.merge(sensor_networks)
merged_earthquakes[['place', 'mag', 'net', 'netName']].tail()

In [ ]:
# Count earthquakes by sensor network name
merged_earthquakes.groupby('netName')['mag'].count()

## Which ten places had the most earthquakes?

In [ ]:
recent_earthquakes['place'].value_counts()[:10]

## Which ten places had the largest earthquakes?

In [ ]:
recent_earthquakes.groupby('place')['mag'].max().sort_values(ascending=False)[:10]

## Where are these earthquakes in the spherical mercator projection?

In [ ]:
from geometryIO import get_transformPoint, proj4LL, proj4SM
from pandas import Series

transform_point = get_transformPoint(proj4LL, proj4SM)

def transform_row(row):
    x, y = transform_point(row['longitude'], row['latitude'])
    return Series(dict(row.to_dict(), **{'x': x, 'y': y}))

recent_earthquakes_xy = recent_earthquakes.apply(transform_row, axis=1)
recent_earthquakes_xy[:3]